[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/CogVideoX-5B-jupyter/blob/main/CogVideoX_5B_jupyter_free.ipynb)

In [ ]:
!pip install xformers==0.0.27.post2 diffusers==0.30.1 accelerate==0.33.0 sentencepiece==0.2.0 moviepy==1.0.3 optimum-quanto==0.2.4 bitsandbytes==0.43.3
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vdo/CogVideoX-5b/raw/main/vae/config.json -d /content/CogVideoX-5b/vae -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vdo/CogVideoX-5b/resolve/main/vae/diffusion_pytorch_model.safetensors -d /content/CogVideoX-5b/vae -o diffusion_pytorch_model.safetensors

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vdo/CogVideoX-5b/raw/main/scheduler/scheduler_config.json -d /content/CogVideoX-5b/scheduler -o scheduler_config.json

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vdo/CogVideoX-5b/raw/main/tokenizer/added_tokens.json -d /content/CogVideoX-5b/tokenizer -o added_tokens.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vdo/CogVideoX-5b/raw/main/tokenizer/special_tokens_map.json -d /content/CogVideoX-5b/tokenizer -o special_tokens_map.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vdo/CogVideoX-5b/resolve/main/tokenizer/spiece.model -d /content/CogVideoX-5b/tokenizer -o spiece.model
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vdo/CogVideoX-5b/raw/main/tokenizer/tokenizer_config.json -d /content/CogVideoX-5b/tokenizer -o tokenizer_config.json

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/cogvideox-5b-float16/raw/main/text_encoder/config.json -d /content/CogVideoX-5b/text_encoder -o config.json 
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/cogvideox-5b-float16/resolve/main/text_encoder/model-00001-of-00003.safetensors -d /content/CogVideoX-5b/text_encoder -o model-00001-of-00003.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/cogvideox-5b-float16/resolve/main/text_encoder/model-00002-of-00003.safetensors -d /content/CogVideoX-5b/text_encoder -o model-00002-of-00003.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/cogvideox-5b-float16/resolve/main/text_encoder/model-00003-of-00003.safetensors -d /content/CogVideoX-5b/text_encoder -o model-00003-of-00003.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/cogvideox-5b-float16/raw/main/text_encoder/model.safetensors.index.json -d /content/CogVideoX-5b/text_encoder -o model.safetensors.index.json

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/cogvideox-5b-float16/raw/main/transformer/config.json -d /content/CogVideoX-5b/transformer -o config.json 
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/cogvideox-5b-float16/resolve/main/transformer/diffusion_pytorch_model-00001-of-00003.safetensors  -d /content/CogVideoX-5b/transformer -o diffusion_pytorch_model-00001-of-00003.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/cogvideox-5b-float16/resolve/main/transformer/diffusion_pytorch_model-00002-of-00003.safetensors  -d /content/CogVideoX-5b/transformer -o diffusion_pytorch_model-00002-of-00003.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/cogvideox-5b-float16/resolve/main/transformer/diffusion_pytorch_model-00003-of-00003.safetensors  -d /content/CogVideoX-5b/transformer -o diffusion_pytorch_model-00003-of-00003.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/cogvideox-5b-float16/raw/main/transformer/diffusion_pytorch_model.safetensors.index.json  -d /content/CogVideoX-5b/transformer -o diffusion_pytorch_model.safetensors.index.json

import torch
import random
import os
import math
import time
from typing import Union, List
import PIL.Image
from datetime import datetime
import numpy as np
from diffusers.image_processor import VaeImageProcessor
from diffusers.utils import export_to_video
import moviepy.editor as mp
from diffusers import AutoencoderKLCogVideoX, CogVideoXTransformer3DModel, CogVideoXDDIMScheduler, CogVideoXPipeline
from transformers import T5EncoderModel, T5Tokenizer

def convert_to_gif(video_path):
    clip = mp.VideoFileClip(video_path)
    clip = clip.set_fps(8)
    clip = clip.resize(height=240)
    gif_path = video_path.replace(".mp4", ".gif")
    clip.write_gif(gif_path, fps=8)
    return gif_path

def save_video(tensor: Union[List[np.ndarray], List[PIL.Image.Image]], fps: int = 8):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    video_path = f"{timestamp}.mp4"
    export_to_video(tensor, video_path, fps=fps)
    return video_path

transformer = CogVideoXTransformer3DModel.from_pretrained("/content/CogVideoX-5b", subfolder="transformer", torch_dtype=torch.float16)
# transformer = CogVideoXTransformer3DModel.from_pretrained("/content/CogVideoX-5b", subfolder="transformer", quantization_config=quantization_config)
print(next(transformer.parameters()).device)
print(next(transformer.parameters()).dtype)

text_encoder = T5EncoderModel.from_pretrained("/content/CogVideoX-5b", subfolder="text_encoder", torch_dtype=torch.float16)
# text_encoder = T5EncoderModel.from_pretrained("THUDM/CogVideoX-5b", subfolder="text_encoder", device_map="auto", quantization_config=quantization_config)
print(next(text_encoder.parameters()).device)
print(next(text_encoder.parameters()).dtype)

vae = AutoencoderKLCogVideoX.from_pretrained("/content/CogVideoX-5b", subfolder="vae", torch_dtype=torch.float16)
print(next(vae.parameters()).device)
print(next(vae.parameters()).dtype)

scheduler = CogVideoXDDIMScheduler.from_pretrained("/content/CogVideoX-5b", subfolder="scheduler")
tokenizer = T5Tokenizer.from_pretrained("/content/CogVideoX-5b", subfolder="tokenizer")

pipe = CogVideoXPipeline(transformer=transformer, text_encoder=text_encoder, vae=vae, tokenizer=tokenizer, scheduler=scheduler)

pipe.enable_model_cpu_offload()
pipe.enable_sequential_cpu_offload()
pipe.vae.enable_slicing()
pipe.vae.enable_tiling()

In [ ]:
prompt = "A golden retriever, sporting sleek black sunglasses, with its lengthy fur flowing in the breeze, sprints playfully across a rooftop terrace, recently refreshed by a light rain. The scene unfolds from a distance, the dog's energetic bounds growing larger as it approaches the camera, its tail wagging with unrestrained joy, while droplets of water glisten on the concrete behind it. The overcast sky provides a dramatic backdrop, emphasizing the vibrant golden coat of the canine as it dashes towards the viewer."
seed = 0

if seed == 0:
    random.seed(int(time.time()))
    seed = random.randint(0, 18446744073709551615)
print(seed)

with torch.inference_mode():
    video_pt = pipe(
        prompt=prompt,
        num_videos_per_prompt=1,
        num_inference_steps=50,
        num_frames=49,
        use_dynamic_cfg=True,
        output_type="pt",
        guidance_scale=7.0,
        generator=torch.Generator(device="cpu").manual_seed(seed),
    ).frames

batch_size = video_pt.shape[0]
batch_video_frames = []
for batch_idx in range(batch_size):
    pt_image = video_pt[batch_idx]
    pt_image = torch.stack([pt_image[i] for i in range(pt_image.shape[0])])

    image_np = VaeImageProcessor.pt_to_numpy(pt_image)
    image_pil = VaeImageProcessor.numpy_to_pil(image_np)
    batch_video_frames.append(image_pil)

video_path = save_video(batch_video_frames[0], fps=math.ceil((len(batch_video_frames[0]) - 1) / 6))
gif_path = convert_to_gif(video_path)